In [1]:
import json

with open('ent_drug_data.json') as f:
    d = json.load(f)

In [2]:
d.keys()

dict_keys(['Acoustic Neuroma', 'Adenoiditis', 'Allergic Rhinitis', 'Eustachian Tube Dysfunction', 'Otosclerosis', 'Hyperacusis', "Otitis Externa (Swimmer's Ear)", 'Cholesteatoma', 'Labyrinthitis', 'Mastoiditis', 'Tinnitus', 'Acute Otitis Media', 'Otitis Media with Effusion', 'Chronic Otitis Media', 'Tympanic Membrane Perforation', 'Ménière’s Disease', 'Benign Paroxysmal Positional Vertigo (BPPV)', 'Acoustic Neuroma (Vestibular Schwannoma)', 'Sudden Sensorineural Hearing Loss', 'Presbycusis (Age-related Hearing Loss)', 'Autoimmune Inner Ear Disease', 'Ear Barotrauma', 'Ramsay Hunt Syndrome'])

In [4]:
d['Acoustic Neuroma'].keys()

dict_keys(['BEVACIZUMAB', 'PANTOPRAZOLE', 'BOTULINUM TOXIN TYPE A', 'BRIGATINIB', 'ELTROMBOPAG OLAMINE', 'ACYCLOVIR', 'ASPIRIN', 'ASPIRIN 325 MG', 'ASPIRIN 81 MG', 'DEXAMETHASONE', 'DEXAMETHASONE 1.5 MG', 'GADODIAMIDE', 'GADOBENATE DIMEGLUMINE', 'GADOBUTROL', 'LEVETIRACETAM', 'WARFARIN', 'APREMILAST', 'CALCIUM CARBONATE', 'CYCLOSPORINE', 'DUPILUMAB', 'ESTRADIOL', 'EVEROLIMUS', 'EVEROLIMUS TABLETS', 'LANSOPRAZOLE', 'LOSARTAN', 'METHOTREXATE', 'METHOTREXATE SODIUM', 'ONABOTULINUMTOXINA', 'SECUKINUMAB', 'SULFAMETHOXAZOLE AND TRIMETHOPRIM', 'ACETAZOLAMIDE', 'ACETAZOLAMIDE SODIUM', 'ACETYLCYSTEINE', 'ADENOSINE', 'ALPRAZOLAM', 'BISOPROLOL FUMARATE', 'CARBOXYMETHYLCELLULOSE SODIUM', 'CARVEDILOL', 'CEPHALEXIN', 'CETIRIZINE HYDROCHLORIDE', 'CLOBAZAM', 'DENOSUMAB', 'DESOGESTREL AND ETHINYL ESTRADIOL AND ETHINYL ESTRADIOL', 'DIAZEPAM', 'DIAZEPAM ORAL', 'DIAZEPAM ORAL SOLUTION (CONCENTRATE)', 'DULOXETINE', 'DULOXETINE HYDROCHLORIDE', 'ERGOCALCIFEROL', 'ESCITALOPRAM', 'ESTRADIOL GEL 0.1%', 'ESTRADI

In [6]:
d['Acoustic Neuroma']['BEVACIZUMAB'].keys()

dict_keys(['indications_and_usage', 'dosage_and_administration', 'dosage_forms_and_strengths', 'contraindications', 'warnings_and_cautions', 'adverse_reactions', 'drug_interactions', 'use_in_specific_populations', 'drug_abuse_and_dependence', 'overdosage', 'description', 'clinical_pharmacology', 'clinical_studies', 'how_supplied'])

In [7]:
import requests
from urllib.parse import quote

# 🔐 Step 1: Get Ticket Granting Ticket (TGT)
def get_tgt(api_key):
    url = "https://utslogin.nlm.nih.gov/cas/v1/api-key"
    data = {"apikey": api_key}
    resp = requests.post(url, data=data)
    if resp.status_code == 201:
        tgt = resp.headers["location"].split("/")[-1]
        print("✅ TGT acquired.")
        return tgt
    raise Exception("Failed to get TGT: " + resp.text)

# 🔑 Step 2: Get Single Use Ticket (ST)
def get_st(tgt):
    service = "http://umlsks.nlm.nih.gov"
    url = f"https://utslogin.nlm.nih.gov/cas/v1/api-key/{tgt}"
    data = {"service": service}
    resp = requests.post(url, data=data)
    if resp.status_code == 200:
        return resp.text
    raise Exception("Failed to get ST: " + resp.text)

# 🔍 Step 3: Get UMLS CUI for disease name
def get_cui(disease_name, st):
    url = f"https://uts-ws.nlm.nih.gov/rest/search/current?string={quote(disease_name)}&ticket={st}"
    resp = requests.get(url)
    results = resp.json()["result"]["results"]
    for item in results:
        if item["rootSource"] == "MSH":  # Optional: prioritize MeSH
            print(f"🧠 Found CUI: {item['ui']} for {disease_name}")
            return item["ui"]
    return results[0]["ui"]

# 🔄 Step 4: Map UMLS CUI to SNOMED CT
def get_snomed_code(cui, st):
    url = f"https://uts-ws.nlm.nih.gov/rest/content/current/CUI/{cui}/codes?ticket={st}"
    resp = requests.get(url)
    for code in resp.json()["result"]:
        if code["rootSource"].startswith("SNOMEDCT"):
            print(f"🔗 Found SNOMED CT Code: {code['code']}")
            return code["code"]
    return None

# 💊 Step 5: Use SNOMED CT to fetch RxClass drug classes
def get_rxclass_drug_classes(snomed_code):
    url = f"https://rxnav.nlm.nih.gov/REST/rxclass/class/byDisease?diseaseId={snomed_code}&diseaseSource=SNOMEDCT&relDirection=destination"
    resp = requests.get(url)
    data = resp.json()
    classes = data.get("rxclassDrugInfoList", {}).get("rxclassDrugInfo", [])
    results = []
    for cls in classes:
        name = cls["rxclassMinConceptItem"]["className"]
        class_id = cls["rxclassMinConceptItem"]["classId"]
        results.append((class_id, name))
    print(f"💊 Found {len(results)} drug classes.")
    return results

# 🧪 Step 6: Get drugs for a drug class
def get_drugs_for_class(class_id):
    url = f"https://rxnav.nlm.nih.gov/REST/rxclass/drugs.json?classId={class_id}&relaSource=ATC"
    resp = requests.get(url)
    drugs = resp.json().get("rxclassDrugInfoList", {}).get("rxclassDrugInfo", [])
    return [d["minConcept"]["name"] for d in drugs]

# 🚀 Main driver
def main():
    api_key = "539b225d-6d95-4e0b-a551-a459f7ce473d"  # Your UMLS API key
    disease = "Adenoiditis"

    tgt = get_tgt(api_key)
    st1 = get_st(tgt)
    cui = get_cui(disease, st1)

    st2 = get_st(tgt)
    snomed_code = get_snomed_code(cui, st2)

    if not snomed_code:
        print("SNOMED CT code not found.")
        return

    classes = get_rxclass_drug_classes(snomed_code)
    if not classes:
        print("No drug classes found.")
        return

    # Show first drug class and fetch drugs for it
    class_id, class_name = classes[0]
    print(f"🔍 Searching drugs for class: {class_name}")
    drugs = get_drugs_for_class(class_id)

    if drugs:
        print("🧾 Drugs found:")
        for drug in drugs:
            print(" -", drug)
    else:
        print("No drugs found for this class.")

if __name__ == "__main__":
    main()


✅ TGT acquired.


KeyError: 'result'